In [1]:
!pip install requests

    100% |████████████████████████████████| 512kB 1.3MB/s 
You are using pip version 8.1.1, however version 8.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
import requests
import numpy as np

In [3]:
response = requests.post('http://httpbin.org/post', files=dict(foo='bar'))

In [2]:
_url = 'https://api.projectoxford.ai/vision/v1/analyses'
_maxNumRetries = 10

In [3]:
_key="7ac55b11055d47f7978dd3f7f9975e6a"

In [4]:
url_base = 'api.projectoxford.ai'

In [7]:
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result

In [9]:
# URL direction to image
urlImage = 'https://oxfordportal.blob.core.windows.net/vision/Analysis/3.jpg'

# Computer Vision parameters
params = { 'visualFeatures' : 'All'} 

headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None

result = processRequest( json, data, headers, params )

In [10]:
result

{u'adult': {u'adultScore': 0.013014058582484722,
  u'isAdultContent': False,
  u'isRacyContent': False,
  u'racyScore': 0.01406136155128479},
 u'categories': [{u'name': u'trans_trainstation', u'score': 0.98828125}],
 u'color': {u'accentColor': u'484B83',
  u'dominantColorBackground': u'Black',
  u'dominantColorForeground': u'Black',
  u'dominantColors': [u'Black'],
  u'isBWImg': False},
 u'faces': [],
 u'imageType': {u'clipArtType': 0, u'lineDrawingType': 0},
 u'metadata': {u'format': u'Jpeg', u'height': 1155, u'width': 1500},
 u'requestId': u'47a95823-cd06-46ac-810c-d23b4f4af08c'}

In [15]:
import cv2

ImportError: dlopen(/Users/nipunbatra/anaconda/lib/python2.7/site-packages/cv2.so, 2): Library not loaded: @rpath/libhdf5.10.dylib
  Referenced from: /Users/nipunbatra/anaconda/lib/libopencv_hdf.3.1.0.dylib
  Reason: Incompatible library version: libopencv_hdf.3.1.dylib requires version 12.0.0 or later, but libhdf5.10.dylib provides version 11.0.0

In [14]:
if result is not None:
    # Load the original image, fetched from the URL
    arr = np.asarray( bytearray( requests.get( urlImage ).content ), dtype=np.uint8 )
    img = cv2.cvtColor( cv2.imdecode( arr, -1 ), cv2.COLOR_BGR2RGB )

    renderResultOnImage( result, img )

    ig, ax = plt.subplots(figsize=(15, 20))
    ax.imshow( img )

NameError: name 'cv2' is not defined

In [16]:
# Load raw image file into memory
pathToFileInDisk = r'/Users/puneet2895/Downloads/banana-tomato-fruit-free-bananas-tomatoes.jpg'
with open( pathToFileInDisk, 'rb' ) as f:
    data = f.read()
    
# Computer Vision parameters
params = { 'visualFeatures' : 'Color,Categories,Tags'} 

headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/octet-stream'

json = None

result = processRequest( json, data, headers, params )


In [17]:
result

{u'categories': [{u'name': u'drink_', u'score': 0.92578125}],
 u'color': {u'accentColor': u'C0A70B',
  u'dominantColorBackground': u'White',
  u'dominantColorForeground': u'Yellow',
  u'dominantColors': [u'White', u'Yellow'],
  u'isBWImg': False},
 u'metadata': {u'format': u'Jpeg', u'height': 426, u'width': 640},
 u'requestId': u'fbdbe81c-c188-4d6a-b540-f0f2586e23c7',
 u'tags': [{u'confidence': 0.894765317440033, u'name': u'indoor'},
  {u'confidence': 0.7579843401908875, u'name': u'yellow'},
  {u'confidence': 0.6157460808753967, u'hint': u'food', u'name': u'fruit'},
  {u'confidence': 0.11018916964530945, u'name': u'vegetable'}]}

In [16]:
from werkzeug import secure_filename
